# # SpringBoard Capstone2: Direct Marketing
## Unit16: Pre-processing and Training Data Development 
> Data from Kaggle.com: https://www.kaggle.com/c/bankdirectmarketing/data 

In [17]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [8]:
market_pre = pd.read_csv('Market_explored.csv', index_col = 0)
print(market_pre.head())

          age           job   marital            education  default housing  \
RecordID                                                                      
13783      49        admin.  divorced  professional.course       no     yes   
23986      52      services   married          high.school  unknown     yes   
20663      46   blue-collar  divorced             basic.9y       no      no   
13958      26  entrepreneur    single          high.school      yes     yes   
28184      47        admin.    single    university.degree       no      no   

         loan   contact month day_of_week  ...  campaign  pdays  previous  \
RecordID                                   ...                              
13783     yes  cellular   aug         mon  ...         1    115         2   
23986      no  cellular   may         mon  ...         1    402         2   
20663      no  cellular   apr         wed  ...         1    999         1   
13958     yes  cellular   aug         fri  ...        28    9

In [9]:
cat = market_pre[['job', 'marital', 'education', 'default', 'housing', \
 'loan', 'contact', 'month', 'day_of_week', 'poutcome']]
market_cat = pd.get_dummies(cat,drop_first = True)
print(market_cat.shape)
market_cat.head()

(22500, 43)


,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
RecordID,,,,,,,,,,,,,,,,,,,,,
13783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
23986,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
20663,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13958,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
28184,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


In [10]:
market_num = market_pre[['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', \
             'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']]
market_all = market_cat.join(market_num)
market_all.head()

,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,...,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
RecordID,,,,,,,,,,,,,,,,,,,,,
13783,0,0,0,0,0,0,0,0,0,0,...,49,4457,1,115,2,1.4,92.479703,-35.498996,0.705058,4990.198481
23986,0,0,0,0,0,0,1,0,0,0,...,52,4797,1,402,2,-1.8,93.439161,-39.331320,4.245479,5144.563621
20663,1,0,0,0,0,0,0,0,0,0,...,46,169,1,999,1,-1.8,93.075000,-47.100000,1.445000,5099.100000
13958,0,1,0,0,0,0,0,0,0,0,...,26,376,28,999,0,1.4,93.444000,-36.100000,4.967000,5228.100000
28184,0,0,0,0,0,0,0,0,0,0,...,47,3033,1,252,4,-3.4,94.352376,-33.073620,1.208702,5025.420036


In [15]:
X = market_all
y = market_pre['subscribe']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print (X_train.shape, y_test.shape)

(18000, 53) (4500,)


In [20]:
# build scaler based on training data and apply it to test data to then also scale the test data
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)
X_train_scaled[0:5]

array([[-0.52945128, -0.19874868, -0.17281816, -0.27540376, -0.21707238,
        -0.19478852, -0.31703876, -0.15902987, -0.42956052, -0.17349889,
        -0.10917027, -1.19218602,  1.60445171, -0.12058924, -0.2515238 ,
        -0.41245953, -0.53460839, -0.08529222, -0.37978611,  1.56132595,
        -0.2163699 , -0.52618193, -0.13194316, -0.20397644, -1.03069304,
        -0.20456672, -0.44021845, -0.76526053,  2.38047614, -0.09380282,
        -0.450965  , -0.3785402 , -0.13875355, -0.68993823, -0.33209739,
        -0.14705231, -0.13302862,  1.94870248, -0.51428488, -0.49374193,
        -0.49591674, -1.35647065,  2.34033118, -0.5749672 ,  0.66676257,
        -0.25800036,  0.28764156,  0.87868221,  0.83815053, -0.81298691,
        -0.4222293 ,  0.53345923, -1.95168948],
       [-0.52945128, -0.19874868, -0.17281816, -0.27540376, -0.21707238,
        -0.19478852, -0.31703876, -0.15902987, -0.42956052, -0.17349889,
        -0.10917027,  0.83879527, -0.62326588, -0.12058924, -0.2515238 ,
   